**ANLP LAB 7**


Learning Objectives

1. Understand the Deep Neural Network concept for Language Modelling
2. A simple model to explain how a model can be saved after training and loaded later when it is needed
3. Simple approach to write down a generate function to generate data from learned model with the use of encoder.

We will use Google Colab to write program. We will use Keras backend will be running in TensorFlow

In [ ]:
from google.colab import files
files.upload()

Saving train.txt to train.txt


{'train.txt': b"BOOK I.\r\n\r\nI went down yesterday to the Piraeus with Glaucon the son of Ariston,\r\nthat I might offer up my prayers to the goddess (Bendis, the Thracian\r\nArtemis.); and also because I wanted to see in what manner they would\r\ncelebrate the festival, which was a new thing. I was delighted with the\r\nprocession of the inhabitants; but that of the Thracians was equally,\r\nif not more, beautiful. When we had finished our prayers and viewed the\r\nspectacle, we turned in the direction of the city; and at that instant\r\nPolemarchus the son of Cephalus chanced to catch sight of us from a\r\ndistance as we were starting on our way home, and told his servant to\r\nrun and bid us wait for him. The servant took hold of me by the cloak\r\nbehind, and said: Polemarchus desires you to wait.\r\n\r\nI turned round, and asked him where his master was.\r\n\r\nThere he is, said the youth, coming after you, if you will only wait.\r\n\r\nCertainly we will, said Glaucon; and in a 

In [ ]:
import string
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 

In [ ]:
# load document
in_filename = 'train.txt'
doc = load_doc(in_filename)
print(doc[:200])
 
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))
 
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'train_sequences.txt'
save_doc(sequences, out_filename)

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what
['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'c

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load
in_filename = 'train_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [ ]:
lines = lines[0:500]

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=32, epochs=50)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 50)            11800     
_________________________________________________________________
lstm_7 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 236)               23836     
Total params: 186,536
Trainable params: 186,536
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
500/500 [==============================] - 5s 10ms/step - loss: 5.4484 - acc: 0.0320
Epoch 2/50
50

In [ ]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [ ]:
# load cleaned text sequences
in_filename = 'train_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
lines = lines[0:5000]
seq_length = len(lines[0].split()) - 1

In [ ]:
# load the model
model = load_model('model.h5')

In [ ]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

exceptions suppose that a friend when in his right mind has deposited arms with me and he asks for them when he is not in his right mind ought i to give them back to him no one would say that i ought or that i should be right in doing



In [ ]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

him the alternative but we are we the alternative but we are we the alternative i said you we see rejoined since we said we the alternative i said you we see rejoined since we said we the alternative i said you we see rejoined since we said we the
